In [24]:
import requests
from bs4 import BeautifulSoup

r = requests.get("http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})

c = r.content

soup = BeautifulSoup(c,"html.parser")

extract = soup.find_all("div",{"class":"propertyRow"})

In [72]:
prices = []
for item in extract:
    d={}
    d["Address"] = item.find_all("span",{"class":"propAddressCollapse"})[0].text
    d["Locality"] = item.find_all("span",{"class":"propAddressCollapse"})[1].text
    d["Price"] = item.find("h4",{"class":"propPrice"}).text.strip()
    #Calculate number of beds
    try:
        d["Beds"] = (item.find("span",{"class":"infoBed"}).find("b").text)
    except AttributeError:
        d["Beds"] = None

    #Calculate number of full baths
    try:
        d["Full Baths"] = (item.find("span",{"class":"infoValueFullBath"}).find("b").text)
    except AttributeError:
        d["Full Baths"] = None

    #Calculate number of half baths
    try:
        d["Half Baths"] = (item.find("span",{"class":"infoValueHalfBath"}).find("b").text)
    except AttributeError:
        d["Half Baths"] = None
    
    #Calculate number of Sq. Ft. 
    try:
        d["Area"] = (item.find("span",{"class":"infoSqFt"}).find("b").text)
    except AttributeError:
        d["Area"] = None
    
    for column_group in item.find_all("div",{"class":"columnGroup"}):
        for feature_group, feature_name in zip(column_group.find_all("span",{"class":"featureGroup"}),column_group.find_all("span",{"class":"featureName"})):
            if "Lot Size" in feature_group.text:
                d["Lot Size"] = feature_name.text
            
    prices.append(d)
    

In [73]:
import pandas
df_prices = pandas.DataFrame(prices)

In [74]:
df_prices

,Address,Locality,Price,Beds,Full Baths,Half Baths,Area,Lot Size
0,0 Gateway,"Rock Springs, WY 82901","$725,000",None,None,None,None,NaN
1,1003 Winchester Blvd.,"Rock Springs, WY 82901","$452,900",4,4,None,None,0.21 Acres
2,600 Talladega,"Rock Springs, WY 82901","$396,900",5,3,None,"3,154",NaN
3,3239 Spearhead Way,"Rock Springs, WY 82901","$389,900",4,3,1,"3,076","Under 1/2 Acre,"
4,522 Emerald Street,"Rock Springs, WY 82901","$254,000",3,3,None,"1,172","Under 1/2 Acre,"
5,1302 Veteran's Drive,"Rock Springs, WY 82901","$252,900",4,2,None,"1,932",0.27 Acres
6,1021 Cypress Cir,"Rock Springs, WY 82901","$210,000",4,3,None,"1,676","Under 1/2 Acre,"
7,913 Madison Dr,"Rock Springs, WY 82901","$209,000",3,2,None,"1,344","Under 1/2 Acre,"
8,1344 Teton Street,"Rock Springs, WY 82901","$199,900",3,2,None,"1,920","Under 1/2 Acre,"
9,4 Minnies Lane,"Rock Springs, WY 82901","$196,900",3,2,None,"1,664",2.02 Acres


In [80]:
df_prices.to_excel("RealEstate.xlsx")